# Patch-clamp: Voltage sag

In this tutorial, I explain how to calculate voltage sag from current clamp recordings using custom functions. To read the full tutorial, please see [Patch-clamp data analysis in Python: passive membrane properties](https://spikesandbursts.wordpress.com/2022/05/13/patch-clamp-data-analysis-python-passive-membrane-properties/) of the [Spikes and Bursts](https://spikesandbursts.wordpress.com/) blog.

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import pyabf
import matplotlib.pyplot as plt
from scipy import signal

# Create the paths

In [ ]:
notebook_name = 'voltage_sag'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data':  f'{data_path}/Data',
         'processed_data': f'{data_path}/Processed_data/{notebook_name}',
         'analysis': f'{data_path}/Analysis/{notebook_name}'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Function

In [ ]:
def voltage_sag(time, current, voltage, 
                baseline_window_start=0, baseline_window_end=100, 
                sag_window_start=250, sag_window_end=400, 
                steady_window_start=600, steady_window_end=700):
    
    """
    Function to calculate voltage sag from whole-cell patch clamp recordings. 
    
    Args:
    - time: Array of time points.
    - current: Array of current measurements.
    - voltage: Array of voltage measurements.
    - baseline_window_start: Start time for the baseline voltage window (in ms).
    - baseline_window_end: End time for the baseline voltage window (in ms).
    - sag_window_start: Start time for the sag analysis window (in ms).
    - sag_window_end: End time for the sag analysis window (in ms).
    - steady_window_start: Start time for the steady voltage window (in ms).
    - steady_window_end: End time for the steady voltage window (in ms).

    Returns:
    DataFrame containing sag analysis results.
    """
    
    # Initialize variables to avoid errors when sag peak is not found
    sag_peak_time = None
    sag_peak_index = None
    
    # Current step 
    current_step = np.mean(current[int(steady_window_start * abf.dataPointsPerMs):int(steady_window_end * abf.dataPointsPerMs)])
    
    if current_step < 0:  # Only consider voltage responses to negative current pulses

        # Voltage steady response
        voltage_evoked_steady = np.mean(voltage[int(steady_window_start * abf.dataPointsPerMs):int(steady_window_end * abf.dataPointsPerMs)])

        # Voltage baseline
        voltage_baseline = np.mean(voltage[int(baseline_window_start * abf.dataPointsPerMs):int(baseline_window_end * abf.dataPointsPerMs)])

        # Voltage sag peak amplitude (averaging 10 ms around the peak)
        sag_peak_window_index = np.argmin(voltage[int(sag_window_start * abf.dataPointsPerMs):int(sag_window_end * abf.dataPointsPerMs)])
        sag_peak_start = int(sag_peak_window_index - 5 * abf.dataPointsPerMs) + int(sag_window_start * abf.dataPointsPerMs)
        sag_peak_end = int(sag_peak_window_index + 5 * abf.dataPointsPerMs) + int(sag_window_start * abf.dataPointsPerMs)
        sag_peak_voltage = np.mean(voltage[sag_peak_start:sag_peak_end])
        sag_peak_index = sag_peak_window_index + int(sag_window_start * abf.dataPointsPerMs)
        sag_peak_time = time[sag_peak_index]

        # Sag amplitude
        voltage_sag_amplitude = np.abs(sag_peak_voltage - voltage_evoked_steady)

        # Sag ratio
        sag_ratio = (voltage_sag_amplitude / (np.abs(voltage_baseline - sag_peak_voltage))) * 100

        # Add data to the table
        length = len(table_sag)
        table_sag.loc[length, 'Filename'] = basename
        table_sag.loc[length, 'Current_step'] = current_step
        table_sag.loc[length, 'Baseline_voltage'] = voltage_baseline
        table_sag.loc[length, 'Sag_peak_voltage'] = sag_peak_voltage
        table_sag.loc[length, 'Sag_peak_index'] = sag_peak_index
        table_sag.loc[length, 'Sag_peak_time'] = sag_peak_time
        table_sag.loc[length, 'Steady voltage'] = voltage_evoked_steady
        table_sag.loc[length, 'Sag_amplitude_voltage'] = voltage_sag_amplitude
        table_sag.loc[length, 'Sag_ratio'] = sag_ratio

    return table_sag, sag_peak_time, sag_peak_index

# Example data

In this notebook, I show an example of how to simply loop through files with the common filename prefix **pfc_pyr_aps** within the **Data** folder or the path you define. 

# Define the parameters

In [ ]:
# Voltage sag analysis parameters
baseline_window_start = 0
baseline_window_end = 150 
sag_window_start = 250
sag_window_end = 400
steady_window_start = 600
steady_window_end = 700

# Cursors in ms
cursors_ms = [
    baseline_window_start, baseline_window_end,
    sag_window_start, sag_window_end,
    steady_window_start, steady_window_end
]

# Visualize the cursors in one recording
filename = "pfc_pyr_aps"

data_path = f"{paths['data']}/{filename}_04.abf" 
abf = pyabf.ABF(data_path)
print(abf)

fig, ax = plt.subplots(figsize=(6, 4))

# Plot sweeps
for sweepNumber in abf.sweepList:
    abf.setSweep(sweepNumber)
    ax.plot(abf.sweepX*1000, abf.sweepY)  # convert to ms for consistency

# Labels
ax.set_ylabel(abf.sweepLabelY)
ax.set_xlabel('Time (ms)')
ax.set_xlim(0, 1000)
ax.set_ylim(-110, -60)

# Plot cursors
for i, cursor in enumerate(cursors_ms):
    ax.axvline(cursor, linestyle='dotted', color='gray')
    ax.text(cursor, ax.get_ylim()[1], str(i+1),
            ha='center', va='top', color='black')

plt.show()

# Results

In [ ]:
# Prefix filenames
dataset_filenames = 'pfc_pyr_aps'  # For the example data

# Initialize data table
table_sag = pd.DataFrame()

# Iterate through all files in dataset_path
for dirpath, dirnames, filenames in os.walk(paths['data']):
    for filename in filenames:
        
        # Select specific files from your data folder
        if (filename.startswith(dataset_filenames)
            and filename.endswith(".abf")):
            
            try:
                # Paths and names
                file_path = os.path.join(dirpath, filename)
                basename = os.path.splitext(filename)[0]

                # Load the ABF file
                abf = pyabf.ABF(file_path)
                fs = abf.dataPointsPerMs * 1000  # Sampling frequency

                # Bessel lowpass filter (optional)
                b_lowpass, a_lowpass = signal.bessel(4, 1000, 'lowpass', analog=False, fs=fs)

                # Apply filter
                b_combined = b_lowpass
                a_combined = a_lowpass

                # Create the figure and axes
                fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 4), 
                                               sharex=True, gridspec_kw={'height_ratios': [2, 1]})

                for sweepNumber in abf.sweepList:
                    abf.setSweep(sweepNumber)

                    current = abf.sweepC
                    time = abf.sweepX
                    # Apply filtering to voltage signal
                    voltage = signal.filtfilt(b_combined, a_combined, abf.sweepY) 

                    # Initialize variables to avoid errors when sag peak is not found
                    sag_peak_time = None
                    sag_peak_index = None

                    # Get voltage sag calculations
                    table_sag, sag_peak_time, sag_peak_index = voltage_sag(time, current, voltage, 
                                                                           sag_window_start=sag_window_start, 
                                                                           sag_window_end=sag_window_end, 
                                                                           steady_window_start=steady_window_start, 
                                                                           steady_window_end=steady_window_end)

                    if sag_peak_time is not None and sag_peak_index is not None:

                        # Plot the data the traces
                        ax1.plot(time, voltage, linewidth=0.5)
                        ax2.plot(time, current)

                        # Plot sag peaks
                        ax1.plot(sag_peak_time, voltage[sag_peak_index], 'ko', markersize=2)

                # Adjust axes and save the plot
                ax1.set_ylabel(abf.sweepLabelY)
                ax1.axvline(x=sag_window_start/1000, color='grey', linestyle='--', linewidth=0.5)
                ax1.axvline(x=sag_window_end/1000, color='grey', linestyle='--', linewidth=0.5)
                ax1.set_xlim(0, 1.5)
                ax2.set_xlabel(abf.sweepLabelX)
                ax2.set_ylabel(abf.sweepLabelC)

                # Save figure for each recording
                fig.tight_layout()
                fig.savefig(f"{paths['analysis']}/{filename}_voltage_sag.png", dpi=300)  # or svg, tif, etc.
                plt.close()  # Close the plots to save memory

            except Exception as e:
                print(f"Error processing {filename}: {e}")

                
# Save the table 
table_sag.to_csv(f"{paths['analysis']}/{filename}_voltage_sag_results.csv", index=False)
table_sag